In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor
import requests as r

In [2]:
market = Market()
market.connect()
dates = market.retrieve_date_range()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
stuff = pd.to_datetime(dates["date"])

In [4]:
start = (stuff.max() + timedelta(days=1)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2021-12-18 2021-12-21


In [5]:
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        data = TiingoExtractor.extract(ticker,start,end)
        data["ticker"] = ticker
        market.store("prices",data)
        test = pdr.yahoo.daily.YahooDailyReader(symbols=ticker
                                                , start=start
                                                , end=end
                                                , retry_count=3, pause=0.1, session=None, adjust_price=True)
        panda_reader_data = test.read()
        panda_reader_data["ticker"] = ticker
        panda_reader_data.reset_index(inplace=True)
        market.store("pdr_prices",panda_reader_data)
        url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY'
        stuff = r.get(url,params={"apikey":token,"symbol":ticker,"outputsize":"compact"})
        data = stuff.json()
        example = pd.DataFrame.from_records(data["Time Series (Daily)"]).T
        for col in example.columns:
            example.rename(columns={col:col.split(".")[1].strip()},inplace=True)
        example.reset_index(inplace=True)
        example.rename(columns={"index":"date"},inplace=True)
        example["ticker"] = ticker
        market.store("alpha_prices",example)
        sleep(12)
    except Exception as e:
        print(str(e))
market.disconnect()

 16%|████████▉                                              | 82/505 [00:13<01:05,  6.42it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


 53%|████████████████████████████▊                         | 269/505 [00:44<00:53,  4.40it/s]

project_market prices documents must be a non-empty list


 61%|████████████████████████████████▊                     | 307/505 [00:49<00:35,  5.53it/s]

project_market prices documents must be a non-empty list


100%|█████████████████████████████████████████████████████▊| 503/505 [01:21<00:00,  6.94it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment
Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


100%|██████████████████████████████████████████████████████| 505/505 [01:21<00:00,  6.17it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment
Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


In [6]:
data

In [7]:
analysis = []
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_prices("prices",ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        periods = ["year","quarter","month","week"]
        ep = test.tail(1)["adjClose"].item()
        # periods = ["year"]
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(str(e))
market.disconnect()

 13%|███████▏                                               | 66/505 [00:04<00:29, 14.82it/s]

'date'


 16%|████████▋                                              | 80/505 [00:05<00:24, 17.02it/s]

'date'


100%|██████████████████████████████████████████████████████| 505/505 [00:35<00:00, 14.07it/s]

'date'
'date'
'date'
'date'


In [9]:
period = "day"
a.sort_values(f"{period}_gain",ascending=False).head(50)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,Security,GICS Sector
85,-0.072620,-0.275585,0.153846,0.000000,0.033917,CCL,Carnival Corporation,Consumer Discretionary
368,0.707831,0.421384,0.115947,0.000000,0.025891,PFE,Pfizer,Health Care
78,0.365318,0.000000,0.000000,0.008613,0.022518,COG,Cabot Oil & Gas,Energy
27,0.143424,-0.200924,0.062654,0.000000,0.020649,AAL,American Airlines Group,Industrials
341,-0.126001,-0.267397,0.165823,0.000000,0.020167,NCLH,Norwegian Cruise Line Holdings,Consumer Discretionary
135,-0.082146,-0.085678,0.128549,0.000000,0.019643,DVA,DaVita,Health Care
179,0.236431,-0.043348,0.043492,0.000000,0.019584,EXPE,Expedia Group,Consumer Discretionary
367,-0.124487,-0.208608,0.071967,0.000000,0.018823,PRGO,Perrigo,Health Care
164,-0.065387,-0.095202,0.080596,0.000000,0.018821,EA,Electronic Arts,Communication Services
278,-0.212092,-0.059713,0.175358,0.000000,0.017402,LW,Lamb Weston,Consumer Staples


In [10]:
a.groupby("GICS Sector").mean().sort_values("day_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Utilities,0.157176,0.080838,0.069010,0.000000,0.000835
Consumer Staples,0.125446,0.071190,0.082699,0.000000,-0.000863
Communication Services,0.086908,-0.093001,0.036655,0.000000,-0.009259
Health Care,0.195934,0.016938,0.055536,0.000000,-0.009893
Real Estate,0.464875,0.077047,0.053539,0.000000,-0.011562
Information Technology,0.220256,0.033849,0.009483,0.000000,-0.013654
Energy,0.504425,-0.027565,-0.010934,0.000391,-0.015986
Industrials,0.219950,0.014326,0.011316,0.000000,-0.016179
Consumer Discretionary,0.219534,0.001728,0.000610,0.000000,-0.017686
